In [1]:
print("hello")

hello


In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [3]:
## load trained load model
model=load_model("model.h5")

## load encoder and scaler
with open("geo_encoder.pkl","rb") as file:
    geo_encoder=pickle.load(file)

with open("label_encoder.pkl","rb") as file:
    label_encoder=pickle.load(file)

with open("scaler.pkl","rb") as file:
    scaler=pickle.load(file)

In [4]:
input_data={
    'CreditScore':600,	
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [5]:
geo_encoded=geo_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded

c:\Users\Shree\anaconda3\envs\myenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([[1., 0., 0.]])

In [7]:
geo_encoded_df=pd.DataFrame(geo_encoded,columns=geo_encoder.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [11]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [12]:
input_df['Gender']=label_encoder.transform([input_df['Gender']])
input_df

c:\Users\Shree\anaconda3\envs\myenv\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [13]:
input_df=pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [14]:
## scale your data
input_scaled=scaler.transform(input_df)

In [15]:
input_scaled[0]

array([-0.5152381 ,  0.91301738,  0.10285036, -0.68699257, -0.26063048,
        0.79790076,  0.6493972 ,  0.96728484, -0.8682109 ,  0.99850112,
       -0.57812007, -0.57542576])

In [16]:
model.predict(input_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step


array([[0.03957759]], dtype=float32)

In [17]:
prediction_proba=model.predict(input_scaled)[0][0]
prediction_proba

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step


np.float32(0.03957759)

In [18]:
if prediction_proba >0.5:
    print("The customer is likely to churn.")
else:
    print("The coustomer is not likely to churn.")

The coustomer is not likely to churn.
